## __Chapter 05: Untitled__
---
by Daniel Weller & Holger Mann

In [ ]:
data.packages <- c("bigmemory", "data.table", "rgdal", "rgeos")
lapply(data.packages, require, character.only=T); rm(data.packages)

stat.packages <- c("bivariate", "extRemes", "ks", "mev", "mvtnorm")
lapply(stat.packages, require, character.only=T); rm(stat.packages)

if(.Platform$OS.type!="windows"){
  paths <- c("/Volumes/Climate Rasters/", "/Volumes/Analysis Storage", "~/GitHub/Report/")} else {
    paths <- c("D:/Climate Rasters/", "E:/Analysis Storage/", "C:/Users/Cloud/GitHub/Report/")}
setwd(paths[3]); options(scipen=10); set.seed(12345); setDTthreads(16)
rasterOptions(progress="text", maxmemory=1e+09, chunksize=1e+08)

In [ ]:
rtn.years <- seq(10, 50, by=10); heat.rank <- c(0, 50, 100, 155, 225, 300, 400, 525, 700, 875)/1000
cold.times <- big.matrix(nrow=cells, ncol=length(rtn.years), init=NULL, separated=FALSE,
                         backingpath=".", backingfile="Cold_Times_Back", descriptorfile="Cold_Times_Desc")

for(i in 1:length(cold.rank)){
  for(j in cell.list){
    values <- fread(paste(paths[2], "Cold Wave Large/Cell ", j, ".csv",sep=""))
    values <- values[, Rise := round(Thresh, digits=2) - MaxDay][WaveID >= 1]
    values[, `:=`(Excess=sum(round(Rise, digits=2)), Length=max(DaysCount, na.rm=TRUE)), by=.(Year, WaveID)]
    values <- unique(values[, .(Year, WaveID, Excess, Length)])[Excess >= 0.03 & Length >= 3]
    for(k in 1:nrow(values)){values[k, Prob := cold.fun(values[k,3], values[k,4])]}
    values <- values[Prob >= cold.rank[i]][, .(Year, Excess, Length)]
    values[, `:=`(Row=seq_len(.N)), by=.(Year)]; counts <- max(values[, .(Row)])
    values <- values[CJ(Year=1979:2019, Row=1:counts, unique=TRUE), on=.(Year, Row)]
    setnafill(values, fill=0, cols=c("Excess", "Length"))
    try(precs <- NC.diag(values$Length, u=seq(0, quantile(values$Length, probs=0.99), by=1), size=0.1))
    try(which <- min(which(precs$e.p.values > 0.1)))
    try(thrsh <- precs$u[which])
    try(if(is.na(thrsh)){thrsh <- 0})
    try(clust <- decluster(values$Length, threshold=thrsh, method="runs"))
    try(model <- fevd(clust, threshold=thrsh, type="GP", span=years))
    ifelse(exists("model"), cold.times[j,] <- return.level(model, rtn.years)[1:5], cold.times[j,] <- 0)
    try(rm(precs, which, thrsh, clust, model, values), silent=TRUE)
    cat(paste("Current Cell:", j, "in", i, "\n", seq=""))
    if(j%in%seq(1, cells, by=1e+2)){invisible(gc())}}
  write.big.matrix(cold.times, paste("./Cold_Wave_Times_", i,".csv", sep="")); cold.times <- NA}


heat.times <- big.matrix(nrow=cells, ncol=length(rtn.years), init=NULL, separated=FALSE,
                         backingpath=".", backingfile="Heat_Times_Back", descriptorfile="Heat_Times_Desc")

for(i in 1:length(heat.rank)){
  for(j in cell.list){
    values <- fread(paste(paths[2], "Heat Wave Large/Cell ", j, ".csv",sep=""))
    values <- values[, Rise := MaxDay - round(Thresh, digits=2)][WaveID >= 1]
    values[, `:=`(Excess=sum(round(Rise, digits=2)), Length=max(DaysCount, na.rm=TRUE)), by=.(Year, WaveID)]
    values <- unique(values[, .(Year, WaveID, Excess, Length)])[Excess >= 0.03 & Length >= 3]
    for(k in 1:nrow(values)){values[k, Prob := heat.fun(values[k,3], values[k,4])]}
    values <- values[Prob >= heat.rank[i]][, .(Year, Excess, Length)]
    values[, `:=`(Row=seq_len(.N)), by=.(Year)]; counts <- max(values[, .(Row)])
    values <- values[CJ(Year=1979:2019, Row=1:counts, unique=TRUE), on=.(Year, Row)]
    setnafill(values, fill=0, cols=c("Excess", "Length"))
    try(precs <- NC.diag(values$Length, u=seq(0, quantile(values$Length, probs=0.99), by=1), size=0.1))
    try(which <- min(which(precs$e.p.values > 0.1)))
    try(thrsh <- precs$u[which])
    try(if(is.na(thrsh)){thrsh <- 0})
    try(clust <- decluster(values$Length, threshold=thrsh, method="runs"))
    try(model <- fevd(clust, threshold=thrsh, type="GP", span=years))
    ifelse(exists("model"), heat.times[j,] <- return.level(model, rtn.years)[1:5], heat.times[j,] <- 0)
    try(rm(precs, which, thrsh, clust, model, values), silent=TRUE)
    cat(paste("Current Cell:", j, "in", i, "\n", seq=""))
    if(j%in%seq(1, cells, by=1e+2)){invisible(gc())}}
  write.big.matrix(heat.times, paste("./Heat_Wave_Times_", i,".csv", sep="")); heat.times[,] <- NA)}